In [21]:
!pip install transformers 

# # !pip install wget

In [22]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer,AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os 
import re
os.environ["WANDB_DISABLED"] = "true"

In [23]:
import re
def clean(data):
    with open('dataset/teencode.txt','r') as file:
      file = file.read()
      lines = file.split('\n')
      for line in lines:
        elements = line.split('\t')
        data = re.sub(r'\b{}\b'.format(elements[0]), elements[1], data)
    data = re.sub('per ','',data)
    data = re.sub(r'\s+', ' ', data)
    return data
  # print(s)

In [25]:

# call the function
df_train = pd.read_excel('dataset/train.xlsx')
df_test =  pd.read_excel('dataset/test.xlsx')
df_valid = pd.read_excel('dataset/valid.xlsx')

df_train['Sentence'] = df_train['Sentence'].apply(clean)
df_test['Sentence'] = df_test['Sentence'].apply(clean)
df_valid['Sentence'] = df_valid['Sentence'].apply(clean)



train_texts = list(df_train['Sentence'])
test_texts = list(df_test['Sentence'])
valid_texts = list(df_valid['Sentence'])

y= LabelEncoder()

train_labels = y.fit_transform(df_train['Emotion'])
test_labels = y.fit_transform(df_test['Emotion'])
valid_labels = y.fit_transform(df_valid['Emotion'])

target_names = list(df_train.Emotion.unique())


In [26]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)

# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification

model_name = "vinai/phobert-large" 

# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/vinai/phobert-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/44162aafdccde1e25f76f0d54580322fac02b26e8eacb597f24bd6bc187681fd.ef527b5c1bcf4a449629ee818cd749a4d27ce387432d22f53654850fbf98521f
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": 

In [27]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)


# https://huggingface.co/transformers/v3.4.0/custom_datasets.html
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset

train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)
test_dataset = NewsGroupsDataset(test_encodings, test_labels)

label2id = {"Anger": 0, "Disgust": 1, "Enjoyment": 2, "Fear": 3, "Other": 4, "Sadness": 5, "Surprise": 6}
id2label = {0: "Anger", 1: "Disgust", 2: "Enjoyment", 3: "Fear", 4: "Other", 5: "Sadness", 6: "Surprise"}

model = AutoModelForSequenceClassification.from_pretrained(model_name,label2id=label2id,
                        id2label=id2label, num_labels=7).to("cuda")


loading configuration file https://huggingface.co/vinai/phobert-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/44162aafdccde1e25f76f0d54580322fac02b26e8eacb597f24bd6bc187681fd.ef527b5c1bcf4a449629ee818cd749a4d27ce387432d22f53654850fbf98521f
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "Anger",
    "1": "Disgust",
    "2": "Enjoyment",
    "3": "Fear",
    "4": "Other",
    "5": "Sadness",
    "6": "Surprise"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "Anger": 0,
    "Disgust": 1,
    "Enjoyment": 2,
    "Fear": 3,
    "Other": 4,
    "Sadness": 5,
    "Surprise": 6
  },
  "layer_norm_eps": 1e

In [28]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

training_args = TrainingArguments(
    output_dir='my_model',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=4e-5,


)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
# train the model
trainer.train()
# evaluate the current model after training
trainer.evaluate()
# saving the fine tuned model & tokenizer

# model_path = "..output/vinai/phobert-base"
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5548
  Num Epochs = 5
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accu

Step,Training Loss
500,1.515700
1000,1.007000
1500,0.564100


Saving model checkpoint to my_model/checkpoint-500
Configuration saved in my_model/checkpoint-500/config.json
Model weights saved in my_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-500/special_tokens_map.json
added tokens file saved in my_model/checkpoint-500/added_tokens.json
Saving model checkpoint to my_model/checkpoint-1000
Configuration saved in my_model/checkpoint-1000/config.json
Model weights saved in my_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-1000/special_tokens_map.json
added tokens file saved in my_model/checkpoint-1000/added_tokens.json
Saving model checkpoint to my_model/checkpoint-1500
Configuration saved in my_model/checkpoint-1500/config.json
Model weights saved in my_model/checkpoint-1500/pytorch_model.bin
tokenizer config fil

{'eval_loss': 1.4691859483718872,
 'eval_accuracy': 0.6311953352769679,
 'eval_runtime': 5.9762,
 'eval_samples_per_second': 114.789,
 'eval_steps_per_second': 7.697,
 'epoch': 5.0}

In [29]:
from sklearn.metrics import classification_report
import numpy as np
pred = trainer.predict(test_dataset)

# print(test_labels)
y_pred = np.argmax(pred.predictions, axis=1)
# print(y_pred)
print(classification_report(test_labels,y_pred))

***** Running Prediction *****
  Num examples = 693
  Batch size = 15


              precision    recall  f1-score   support

           0       0.53      0.60      0.56        40
           1       0.66      0.59      0.62       132
           2       0.76      0.73      0.74       193
           3       0.67      0.76      0.71        46
           4       0.61      0.67      0.64       129
           5       0.69      0.72      0.70       116
           6       0.75      0.65      0.70        37

    accuracy                           0.68       693
   macro avg       0.67      0.67      0.67       693
weighted avg       0.68      0.68      0.68       693



In [31]:
trainer.save_model('trainer-phobert')

Saving model checkpoint to trainer-phobert
Configuration saved in trainer-phobert/config.json
Model weights saved in trainer-phobert/pytorch_model.bin
tokenizer config file saved in trainer-phobert/tokenizer_config.json
Special tokens file saved in trainer-phobert/special_tokens_map.json
added tokens file saved in trainer-phobert/added_tokens.json
